In [ ]:
"""
【scrapy的使用】


创建scrapy 项目
scrapy startproject project_name 

创建爬虫
scrapy genspider spider_name 允许爬取的范围

运行爬虫 (进入到爬虫所有目录)
scrapy crawl spider_name


使用pipeline
1、实现pipelines.py中的 process_item 方法

2、去配置文件中开启pipeline
ITEM_PIPELINES = {
    'demo01.pipelines.Demo01Pipeline': 300, #数值越大数先执行
}


使用logging
去配置文件中配置

LOG_LEVEL = "WARNING"
LOG_FILE = "./file.log"	#保存路径，设置后不会再在终端中显示日志，需要使用logger


日志的级别有

CRITICAL - 严重错误
ERROR - 一般错误
WARNING - 警告信息
INFO - 一般信息
DEBUG - 调试信息  默认



import logging

logging.warning('your message') 输出格式: 2019-5-20 [root] WARNING:your message，不知道在哪个文件

logger = loggin.getLogger(__name__)
logger.warning('your message') 输出格式: 2019-5-20 [your file] WARNING:your message

# logging模块的设置可以通过loggin.baseConfig()来完成，百度一下参数即可


关于scrapy中实现翻页思路：
1、找到下一页的地址
2、构造一人下一页url地址的request请求传给调度器

next_url = response.xpath('pettern')
while len(next_url) > 0:
    yield scrapy.Request(next_url,callback=self.parse) # yield request对象，就可以交给scrapy引擎



【scrapy.Request知识点】
scrapy.Request(url,[,callback,method="GET",headers,body,cookie,meta,dont_filter=False])
注：中括号中的参数是可选参数
常用参数为：
callback：指定传入的url交给哪个解析函数去处理

meta：实现在不同的解析函数中传递数据，meta默认会携带部分信息，如下载延迟、请求深度等

别的解析函数中取meta
data = response.meta

dont_filter：让scrapy的去重不会过滤当前的url,scrapy默认有url去重功能，对需要重复请求的url有重要用途

注意点：Request已经提供了cookie参数，cookie就不要再放到headers中去了


【pipeline】
pipeline中写入数据到mongodb，记得将item转为字典


【scrapy shell】
scrapy shell 是一个交互终端，我们可以在未启动spider的情况下尝试及调试代码，也可以用来测试xpath

用法：
scrapy shell url

response.url：当前响应的url地址
response.request.url：当前响应的请求的url地址
response.headers：响应头
response.body：响应体，也就是html代码，默认是byte类型
response.request.headers：当前响应的请求头


【setting相关】

settings中可以放一些全局变量供多个地方使用，一般全大写
1、spider 对象里面有settings属性，可直接使用
self.settings['key']
self.settings.get('key','default_value')

2、导入setting
from your_project import settings
print(settings.LOG_LEVEL)

ROBOTSTXT_OBEY = True #是否遵循robots.txt中的规则

CONCURRENT_REQUESTS # 并发数量

DOWNLOAD_DELAY # 下载延迟

COOKIES_ENABLED  # 请求是否带cookie



【pipeline中的方法】

    def __init__(self, mongo_url, db_name):
        self.mongo_url = mongo_url
        self.db_name = db_name

    @classmethod
    def from_crawler(cls, crawler):
        return cls(mongo_url=crawler.settings.get("MONGO_URL"), db_name=crawler.settings.get("DB_NAME"))

    def open_spider(self, spider):
        self.client = MongoClient(self.mongo_url)
        self.db = self.client[self.db_name]
        self.context = self.db["users"]

    def close_spider(self, spider):
        self.client.close()

    def process_item(self, item, spider):
        self.context.update({'url_token': item['url_token']}, dict(item), True)
        return item





"""